# The Fama-French 5 Factor Model

## CAPM

In [1]:
import os

import pandas as pd

In [6]:
TICKER_PATH = '../indices/'

START_DATE = '2000-01-01'
END_DATE = '2019-12-31'

if not os.path.isdir(TICKER_PATH) or len(os.listdir(TICKER_PATH)) == 0:
    %run ../stock_script.py start=$START_DATE end=$END_DATE path=$TICKER_PATH

KeyboardInterrupt: 

In [3]:
df_list = list()
for file in os.listdir(TICKER_PATH):
    temp = pd.read_csv(TICKER_PATH+file)
    temp['ticker'] = file.split('_')[0]
    df_list.append(temp)
    
df = pd.concat(df_list)

In [4]:
%run 


%run:
 Run the named file inside IPython as a program.

Usage::

  %run [-n -i -e -G]
       [( -t [-N<N>] | -d [-b<N>] | -p [profile options] )]
       ( -m mod | file ) [args]

Parameters after the filename are passed as command-line arguments to
the program (put in sys.argv). Then, control returns to IPython's
prompt.

This is similar to running at a system prompt ``python file args``,
but with the advantage of giving you IPython's tracebacks, and of
loading all variables into your interactive namespace for further use
(unless -p is used, see below).

The file is executed in a namespace initially consisting only of
``__name__=='__main__'`` and sys.argv constructed as indicated. It thus
sees its environment as if it were being run as a stand-alone program
(except for sharing global objects such as previously imported
modules). But after execution, the IPython interactive namespace gets
updated with all variables defined in the program (except for __name__
and sys.argv). This allows 

C:\Users\cspa3\AppData\Roaming\Python\Python37\site-packages\IPython\core\magics\execution.py:614: UserWarning: you must provide at least a filename.
  warn('you must provide at least a filename.')
